<a href="https://colab.research.google.com/github/smeev-daniil/smeev-daniil.github.io/blob/master/Autocorrelation_function_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.constants import mu_0
from scipy.constants import physical_constants
import matplotlib.pyplot as plt

proton_mag_mom = physical_constants['proton mag. mom.'][0]

def magnetic_dipole_moment(n, r0):
    """
    Calculating the magnetic dipole moment of a spherical particle containing protons.

    Parameters:
    - n: concentration of protons
    - r0: radius of the spherical particle

    Returns:
    - M: total magnetic dipole moment of the particle
    """
    volume = (4/3) * np.pi * r0**3
    total_protons = n * volume
    M_magnitude = total_protons * proton_mag_mom
    M_vector = np.array([0, 0, M_magnitude])
    return M_vector

def magnetic_field_dipole(r, M):
    """
    Calculating the magnetic field of a dipole at a distance r from it.

    Parameters:
    - r: Position vector relative to the dipole (input as a 3D vector!!!)
    - M: Magnetic dipole moment (3D vector from the previous function)

    Returns:
    - B: Magnetic field vector at the point r
    """
    r_magnitude = np.linalg.norm(r)
    if r_magnitude == 0:
        return np.array([0, 0, 0])  # Avoid division by zero

    r_unit = r / r_magnitude
    dot_product = np.dot(M, r_unit)
    B = (mu_0 / (4 * np.pi * r_magnitude**3)) * (3 * dot_product * r_unit - M)
    return B

def magnetic_field_in_cavity(lattice_constant, M, positions, point):
    """
    Calculating the magnetic field at any point within the cavity formed by 8 spheres.

    Parameters:
    - lattice_constant: Distance between the centers of the spheres, basically 2r0
    - M: Magnetic dipole moment of the particles (3D vector from the 1st function)
    - positions: Positions of the dipoles relative to the cavity center
    - point: The point where the magnetic field is to be calculated (3D vector)

    Center of the cavity is the center of the coordinate plane!

    Returns:
    - B_total: Total magnetic field at the specified point
    """
    B_total = np.array([0.0, 0.0, 0.0])

    for pos in positions:
        r = point - pos  # Vector from the dipole to the point of interest
        B_total += magnetic_field_dipole(r, M)
    return B_total

# Generating random points on a sphere of given radius around the origin
def random_points_on_sphere(radius, num_points=100):
    vec = np.random.randn(num_points, 3)  # Random normal distribution
    vec /= np.linalg.norm(vec, axis=1)[:, np.newaxis]  # Normalize to unit vectors
    return radius * vec  # Scale by the radius



# Parameters
n = 100
r0 = 20*10**(-6)
lattice_constant = 2 * r0

# Magnetic dipole moment of a single particle
M_magnitude = magnetic_dipole_moment(n, r0)
M = np.array([0, 0, M_magnitude])  # Assume dipole moment is along z-axis

# Positions of 8 spheres in a cubic lattice centered around origin
positions = [
    np.array([lattice_constant/2, lattice_constant/2, lattice_constant/2]),
    np.array([-lattice_constant/2, lattice_constant/2, lattice_constant/2]),
    np.array([lattice_constant/2, -lattice_constant/2, lattice_constant/2]),
    np.array([-lattice_constant/2, -lattice_constant/2, lattice_constant/2]),
    np.array([lattice_constant/2, lattice_constant/2, -lattice_constant/2]),
    np.array([-lattice_constant/2, lattice_constant/2, -lattice_constant/2]),
    np.array([lattice_constant/2, -lattice_constant/2, -lattice_constant/2]),
    np.array([-lattice_constant/2, -lattice_constant/2, -lattice_constant/2])
]


"""
Here, the grid of a 1000 of random points within the cavity is defined.

First, '.seed(0)' is used so every time the code is run the program chose
the same set of points, so the results might be reproducible.

Second, number of points is set.

Third, the grid is created within a cube [-a/2; a/2]^3.
"""
np.random.seed(0)
num_points = num_grid_points*10
grid_points = lattice_constant * (np.random.rand(num_points, 3) - 0.5)

# Filtering grid points to keep only those outside all the spheres (in the cavity)
filtered_grid_points = []
for point in grid_points:
    if not is_within_spheres(point, positions, r0):
        filtered_grid_points.append(point)

# Selecting a subset of points for the calculation
filtered_grid_points = np.array(filtered_grid_points[:num_grid_points])

# Creating an array of radius-vector magnitudes
radius_magnitudes = np.linspace(0, lattice_constant, 100)

# Calculating the autocorrelation function for each radius-vector magnitude
autocorrelation_values = []
for r_magnitude in radius_magnitudes:
    if r_magnitude == 0:
        autocorrelation_values.append(1.0)  # Autocorrelation at r=0 is 1
    else:
        autocorr = calculate_autocorrelation_for_radius(r_magnitude, filtered_grid_points, lattice_constant, M, positions, r0)
        autocorrelation_values.append(autocorr)


plt.plot(radius_magnitudes, autocorrelation_values)
plt.xlabel('Radius-Vector Magnitude (m)')
plt.ylabel('Autocorrelation Function')
plt.title('Autocorrelation Function of Magnetic Field in the Cavity')
plt.grid(True)
plt.show()

# Parameters
n = 1e28
r0 = 1e-4
lattice_constant = 2 * r0
num_grid_points = 1000


plot_autocorrelation_function(n, r0)

Autocorrelation of the magnetic field for displacement [4.e-06 0.e+00 0.e+00]: 3.165340932856204e-61 T^2
